In [1]:
import sys
sys.path.append('./build')

In [2]:
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, PointCloud

In [3]:
import numpy as np
np.set_printoptions(suppress=True)

In [4]:
import ppf_registration_spatial_hashing as ppf

In [5]:
model = ppf.load_ply_file('./samples/data/parasaurolophus_low_normals2.ply')

In [6]:
model = ppf.load_ply_file('./samples/data-eirik/NEW_MODEL_NORMALS_CLUSTER_POISSON.ply')

In [7]:
viewer = Viewer()
viewer.add(Grid())
model = ppf.load_ply_file('./samples/data-eirik/NEW_MODEL_NORMALS_CLUSTER_POISSON.ply')
modelvis = PointCloud(model.array()[:3,:].T * 0.001)
# modelvis.position = [0,0,7]
viewer.add(modelvis)
scene = ppf.load_ply_file('./samples/data-eirik/smallbin_1_unordered.ply')
scenevis = PointCloud(scene.array()[:3,:].T * 0.001, color='gray')
# modelvis.position = [0,0,7]
viewer.add(scenevis)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [36]:
tau_d = 0.01
scene_leaf_size = 9
ref_point_df = 5
vote_count_threshold = 0.2

In [37]:
minpt, maxpt = ppf.min_max_3d(model)
model_d_dist = np.max(np.array([maxpt.x - minpt.x,
                                maxpt.y - minpt.y, 
                                maxpt.z - minpt.z])) * tau_d
model_d_dist

1.198036003112793

In [38]:
downsampled_model = ppf.voxelgrid_downsample(model, model_d_dist)
downsampled_scene = ppf.voxelgrid_downsample(scene, scene_leaf_size)

In [39]:
# %%timeit
T = ppf.ppf_registration(downsampled_scene,
                                downsampled_model, 
                                model_d_dist, 
                                ref_point_df, 
                                vote_count_threshold)

In [40]:
T

array([[ -0.99017113,  -0.05190907,   0.12987155, -37.846622  ],
       [ -0.12468067,  -0.09312683,  -0.9878169 , -61.129353  ],
       [  0.06337118,  -0.9943002 ,   0.08573945, 828.4441    ],
       [  0.        ,   0.        ,   0.        ,   1.        ]],
      dtype=float32)

In [42]:
transformed_model = ppf.transform_point_cloud(model, T)
transformed_model.array()[:3,:].T

array([[ 21.157484 , -67.14619  , 838.18317  ],
       [  1.6787167, -29.770424 , 836.2079   ],
       [  6.9886136, -85.30176  , 844.76416  ],
       ...,
       [ 18.785929 , -89.61099  , 814.32104  ],
       [ 20.510178 , -81.840904 , 813.50323  ],
       [-75.1129   , -82.995605 , 835.1839   ]], dtype=float32)

In [44]:
T = ppf.icp(scene, model)
T

array([[   0.29032105,    0.28063005,   -0.9148559 , -102.35224   ],
       [   0.09115554,    0.9435752 ,    0.31836718,   30.673239  ],
       [   0.9525798 ,   -0.17582282,    0.24835844,  793.85443   ],
       [   0.        ,    0.        ,    0.        ,    1.        ]],
      dtype=float32)

In [14]:
transformed_scene_points = (np.linalg.inv(T) @ np.vstack((scene.array()[:3,:], np.ones(scene.width))))[:3,:].T.reshape(-1,3)
# transformed_scene_points

In [15]:
viewer = Viewer()
viewer.add(Grid())
scenevis = PointCloud(transformed_scene_points * 0.001, color='white')
viewer.add(scenevis)
modelvis = PointCloud(model.array()[:3,:].T * 0.001, color='red')
viewer.add(modelvis)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…